In [12]:
import pandas as pd
import nltk
from nltk.corpus import brown
import jieba
import gensim
%matplotlib inline

from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [3]:
df = pd.read_csv(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\金庸-射雕英雄传txt精校版.txt", names=["txt"], sep='aaa', encoding="gbk")

# 构建新列
print(len(df))
print(df.shape)
def m_head(tmpstr):
    return tmpstr[:1]

def m_mid(tmpstr):
    return tmpstr.find("回 ")

df["head"] = df.txt.apply(m_mid)
df["mid"] = df.txt.apply(m_head)
df["len"] = df.txt.apply(len)

# 标明章序号
chapnum = 0
for i in range(len(df)):
    if df["mid"][i] == "第" and df["head"][i] > 0 and df["len"][i] < 30:
        chapnum += 1
    if chapnum >= 40 and df.txt[i] == "附录一：成吉思汗家族":
        chapnum = 0
    df.loc[i, "chap"] = chapnum

dfgrp = df.groupby("chap")
chapter = dfgrp.apply(sum)
chapter = chapter[chapter.index != 0]
chapter.head()

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


7165
(7165, 1)


,txt,head,mid,len,chap
chap,,,,,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0


In [5]:
def m_cut(intxt):
    return [w for w in jieba.cut(intxt) if w not in stoplist and len(w) > 1]

In [9]:
stoplistdf = pd.read_csv(r'D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\停用词.txt', names=["w"], sep="aaa", encoding="utf-8")
stoplist = list(stoplistdf.w)

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [14]:
chaplist = [m_cut(w) for w in chapter.txt]

countvec = CountVectorizer(min_df=5)
wordmtx = countvec.fit_transform(chaplist[0])

transformer = TfidfTransformer()
tfidf = transformer.fit_transform(wordmtx)
tfidf

<7131x195 sparse matrix of type '<class 'numpy.float64'>'
	with 2221 stored elements in Compressed Sparse Row format>

In [15]:
import pyLDAvis
import pyLDAvis.sklearn

n_topics = 10

ldamodel = LatentDirichletAllocation(n_components=n_topics)
ldamodel.fit(wordmtx)

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(ldamodel, tfidf, countvec)

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      12.361549        1       1 -0.422026 -0.015904
6      11.235930        1       2  0.064365 -0.405672
7      10.650608        1       3  0.050218  0.121508
1       9.997749        1       4  0.048628  0.051500
9       9.952724        1       5  0.044914  0.052755
8       9.750113        1       6  0.048140  0.045788
5       9.187303        1       7  0.040315  0.039359
0       9.142278        1       8  0.043264  0.038550
3       8.939667        1       9  0.042153  0.036682
4       8.782080        1      10  0.040029  0.035435, topic_info=    Category        Freq Term       Total  loglift  logprob
129  Default  125.000000  杨铁心  125.000000  30.0000  30.0000
52   Default   91.000000  包惜弱   91.000000  29.0000  29.0000
180  Default   62.000000  郭啸天   62.000000  28.0000  28.0000
24   Default   38.000000  丘处机   38.000000  27.0000  27.0000
134  Default   39.000000   武官   39.000000  26.0000  26.0000
..       ...         ...  ...         ...      ...      ...
126  Topic10    0.104027  杨再兴    5.688335  -1.5691  -7.5364
32   Topic10    0.104027   举起    5.688335  -1.5691  -7.5364
152  Topic10    0.104027   老虎    6.480687  -1.6995  -7.5364
1    Topic10    0.104027   一人    6.480687  -1.6995  -7.5364
39   Topic10    0.104027   今日    8.065391  -1.9182  -7.5364

[395 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.950950    一个
1         1  0.925828    一人
2         9  0.897469    一会
3         6  0.932019    一口
4         5  0.938240    一只
...     ...       ...   ...
190       3  0.961238    颜烈
191       1  0.878992    饮酒
192       3  0.808908    马背
193       7  0.847610  高宗皇帝
194       8  0.883979   黑衣人

[195 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'ylab': 'PC2', 'xlab': 'PC1'}, topic_order=[3, 7, 8, 2, 10, 9, 6, 1, 4, 5])

# 文档相似度

In [16]:
chapter.head()

,txt,head,mid,len,chap
chap,,,,,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0


In [18]:
chapter["cut"] = chapter.txt.apply(m_cut)
chapter.head()

,txt,head,mid,len,chap,cut
chap,,,,,,
1.0,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0,"[第一回, 风雪, 惊变, 钱塘江, 浩浩, 江水, 日日夜夜, 无穷, 无休, 浙西, 临..."
2.0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0,"[第二回, 江南七怪, 颜烈, 跨出, 房门, 过道, 一个, 中年, 士人, 直响, 一路..."
3.0,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0,"[第三回, 黄沙, 莽莽, 寺里, 僧众, 焦木, 圆寂, 悲哭, 伤者, 包扎, 伤处, ..."
4.0,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0,"[第四回, 黑风双, 完颜洪熙, 痛快, 蒙古, 前哨, 报来, 王罕, 前来, 迎接, 大..."
5.0,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0,"[第五回, 弯弓, 射雕, 一行, 人下, 山来, 不多时, 猛兽, 吼声, 一阵阵, 传来..."


In [19]:
from gensim.models.word2vec import Word2Vec

n_dim = 300  # 向量维度
w2vmodel = Word2Vec(size=n_dim, min_count=10)
w2vmodel.build_vocab(chapter.cut)  # 生成词
w2vmodel

In [21]:
%time w2vmodel.train(chapter.cut, total_examples=w2vmodel.corpus_count, epochs=10)

Wall time: 1.22 s


(1423905, 2334060)

In [22]:
print(w2vmodel.wv["郭靖"].shape)

(300,)


In [23]:
w2vmodel.wv["郭靖"]

array([-0.19200732, -0.02818786, -0.00945237,  0.04371789,  0.23380436,
       -0.10659672, -0.2699939 ,  0.24773662, -0.41570726, -0.10643582,
       -0.03130645, -0.03687059,  0.05928604,  0.02574679, -0.0458854 ,
        0.23301522, -0.05263896,  0.06659176, -0.11652177,  0.02960725,
        0.14687382, -0.09452423, -0.09022675, -0.0075278 ,  0.1616974 ,
       -0.03898695, -0.1659049 , -0.26902413,  0.03860414,  0.15914594,
        0.07924999, -0.1272441 , -0.00446936,  0.14706828,  0.19863822,
       -0.03330586, -0.05259496, -0.23921439, -0.05631043, -0.04330504,
       -0.01342402,  0.24007663,  0.0651833 , -0.19116561, -0.2593479 ,
        0.17679214,  0.09356035,  0.05779623, -0.02505727, -0.03342227,
       -0.10242905,  0.11994215,  0.15908067, -0.00937913, -0.17112161,
        0.0651268 ,  0.1219154 , -0.09507861, -0.04676998,  0.1236202 ,
        0.19830155,  0.04974933,  0.19357955, -0.00796683,  0.0703913 ,
        0.3235984 ,  0.37757608,  0.04664527,  0.05471514, -0.06

In [24]:
w2vmodel.wv.most_similar("郭靖")

[('再也', 0.9848575592041016),
 ('随即', 0.9836472868919373),
 ('郭靖大', 0.983141303062439),
 ('慢慢', 0.9830657839775085),
 ('一扯', 0.9827969074249268),
 ('摇晃', 0.9825127720832825),
 ('放开', 0.9821613430976868),
 ('郭靖见', 0.9820979833602905),
 ('拉住', 0.9817107915878296),
 ('摔倒', 0.9815869927406311)]

In [27]:
w2vmodel.wv.most_similar("黄蓉")

[('郭靖', 0.9546180963516235),
 ('怀里', 0.9487938284873962),
 ('取出', 0.9468449950218201),
 ('再也', 0.9467921257019043),
 ('一只', 0.9455889463424683),
 ('怀中', 0.9448623657226562),
 ('转身', 0.9445761442184448),
 ('脸上', 0.9403902292251587),
 ('桌上', 0.9396740794181824),
 ('满脸', 0.9396699666976929)]

In [28]:
w2vmodel.wv.most_similar("黄蓉", topn=20)

[('郭靖', 0.9546180963516235),
 ('怀里', 0.9487938284873962),
 ('取出', 0.9468449950218201),
 ('再也', 0.9467921257019043),
 ('一只', 0.9455889463424683),
 ('怀中', 0.9448623657226562),
 ('转身', 0.9445761442184448),
 ('脸上', 0.9403902292251587),
 ('桌上', 0.9396740794181824),
 ('满脸', 0.9396699666976929),
 ('慢慢', 0.9393025636672974),
 ('正要', 0.9386663436889648),
 ('郭靖见', 0.9383264780044556),
 ('月光', 0.9374004602432251),
 ('放在', 0.9373477697372437),
 ('黄蓉见', 0.9372785687446594),
 ('回头', 0.9365932941436768),
 ('郭靖大', 0.9362001419067383),
 ('那人', 0.9352591037750244),
 ('翻身', 0.9352556467056274)]

In [29]:
w2vmodel.wv.most_similar("黄蓉道")

[('郭靖道', 0.9947978854179382),
 ('爹爹', 0.9899015426635742),
 ('问道', 0.9685788154602051),
 ('说道', 0.9683468341827393),
 ('叫化', 0.9561871886253357),
 ('老毒物', 0.9473867416381836),
 ('老人家', 0.9456011056900024),
 ('岛上', 0.9454290270805359),
 ('靖哥哥', 0.944232702255249),
 ('兄弟', 0.9430264234542847)]

In [30]:
w2vmodel.wv.most_similar(["郭靖", "小红马"], ["黄药师"], topn=5)

[('一阵', 0.9764158725738525),
 ('一个', 0.9629660844802856),
 ('脸上', 0.9599467515945435),
 ('两人', 0.9546177983283997),
 ('一块', 0.9530388712882996)]

In [31]:
w2vmodel.wv.most_similar(positive=['郭靖', '黄蓉'], negative=['杨康'], topn=10)

[('一掌', 0.9119493961334229),
 ('手掌', 0.9002276659011841),
 ('跟着', 0.8908655643463135),
 ('右掌', 0.8819581270217896),
 ('左掌', 0.8812803626060486),
 ('肩头', 0.8812122941017151),
 ('一拳', 0.8795270919799805),
 ('劲力', 0.8794403076171875),
 ('竹棒', 0.8776550889015198),
 ('伸出', 0.8775699138641357)]

In [32]:
# 计算两个词的相似度/相关程度
print(w2vmodel.wv.similarity("郭靖", "黄蓉"))
print(w2vmodel.wv.similarity("郭靖", "杨康"))
print(w2vmodel.wv.similarity("郭靖", "杨铁心"))

0.95461816
0.72923756
0.83251774


In [33]:
w2vmodel.wv.doesnt_match("小红马 黄药师 鲁有脚".split())

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'黄药师'

In [34]:
w2vmodel.wv.doesnt_match("杨铁心 黄药师 黄蓉 洪七公".split())

'杨铁心'

## 文档相似度 基于词袋模型

In [36]:
cleanchap = [" ".join(m_cut(w)) for w in chapter.txt.iloc[:5]]

from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()

resmtx = countvec.fit_transform(cleanchap)
resmtx

<5x11684 sparse matrix of type '<class 'numpy.int64'>'
	with 16982 stored elements in Compressed Sparse Row format>

In [43]:
from sklearn.metrics.pairwise import pairwise_distances

pairwise_distances(resmtx, metric ="cosine")

array([[0.        , 0.63250402, 0.77528382, 0.78540047, 0.82880469],
       [0.63250402, 0.        , 0.62572437, 0.61666388, 0.73192845],
       [0.77528382, 0.62572437, 0.        , 0.51645443, 0.5299046 ],
       [0.78540047, 0.61666388, 0.51645443, 0.        , 0.42108002],
       [0.82880469, 0.73192845, 0.5299046 , 0.42108002, 0.        ]])

In [44]:
pairwise_distances(resmtx)

array([[  0.        , 295.77356204, 317.52637686, 320.33576135,
        316.85170033],
       [295.77356204,   0.        , 266.95130642, 265.77622166,
        277.24898557],
       [317.52637686, 266.95130642,   0.        , 233.9615353 ,
        226.09290126],
       [320.33576135, 265.77622166, 233.9615353 ,   0.        ,
        202.57344347],
       [316.85170033, 277.24898557, 226.09290126, 202.57344347,
          0.        ]])

In [45]:
pairwise_distances(tfidf[:5], metric="cosine")

array([[0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 1.],
       [1., 1., 0., 1., 1.],
       [1., 1., 1., 0., 1.],
       [1., 1., 1., 1., 0.]])

In [46]:
tfidf[:5]

<5x195 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

## gensim 相似度实现

In [50]:
chaplist = [m_cut(w) for w in chapter.txt]

dictionary = gensim.corpora.Dictionary(chaplist)  
corpus = [dictionary.doc2bow(text) for text in chaplist] # 仍为list in list  

tfidf_model = gensim.models.TfidfModel(corpus) # 建立TF-IDF模型  
corpus_tfidf = tfidf_model[corpus] # 对所需文档计算TF-IDF结果
corpus_tfidf

In [53]:
# 列出所消耗的时间备查
%time ldamodel = gensim.models.LdaModel(corpus, id2word = dictionary, num_topics = 10, passes = 2) 

Wall time: 2.13 s


In [51]:
simmtx = gensim.similarities.MatrixSimilarity(corpus)
simmtx

In [52]:
simmtx.index[:2]

array([[0.00360987, 0.11551594, 0.00360987, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.15784042, 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [54]:
query = chapter.txt[1]
query_bow = dictionary.doc2bow(m_cut(query))

lda_vec = ldamodel[query_bow]
sims = simmtx[lda_vec]
sims = sorted(enumerate(sims), key=lambda item: -item[1])
sims

[(0, 0.0036098731),
 (1, 0.0),
 (2, 0.0),
 (3, 0.0),
 (4, 0.0),
 (5, 0.0),
 (6, 0.0),
 (7, 0.0),
 (8, 0.0),
 (9, 0.0),
 (10, 0.0),
 (11, 0.0),
 (12, 0.0),
 (13, 0.0),
 (14, 0.0),
 (15, 0.0),
 (16, 0.0),
 (17, 0.0),
 (18, 0.0),
 (19, 0.0),
 (20, 0.0),
 (21, 0.0),
 (22, 0.0),
 (23, 0.0),
 (24, 0.0),
 (25, 0.0),
 (26, 0.0),
 (27, 0.0),
 (28, 0.0),
 (29, 0.0),
 (30, 0.0),
 (31, 0.0),
 (32, 0.0),
 (33, 0.0),
 (34, 0.0),
 (35, 0.0),
 (36, 0.0),
 (37, 0.0),
 (38, 0.0),
 (39, 0.0)]

## 相似度 word2vec

In [56]:
def m_doc(doclist):
    reslist = []
    for i, doc in enumerate(doclist):
        reslist.append(gensim.models.doc2vec.TaggedDocument(jieba.lcut(doc), [i]))
    return reslist

corp = m_doc(chapter.txt)
corp[:2]

[TaggedDocument(words=['第一回', ' ', '风雪', '惊变', '钱塘江', '浩浩', '江水', '，', '日日夜夜', '无穷', '无休', '的', '从', '两', '浙西', '路', '临安', '府', '牛家村', '边', '绕过', '，', '东', '流入', '海', '。', '江畔', '一排', '数十株', '乌', '柏树', '，', '叶子', '似', '火烧', '般红', '，', '正是', '八月', '天时', '。', '村前村后', '的', '野草', '刚', '起始', '变黄', '，', '一抹', '斜阳', '映照', '之下', '，', '更增', '了', '几分', '萧索', '。', '两株', '大', '松树', '下', '围着', '一堆', '村民', '，', '男男女女', '和', '十几个', '小孩', '，', '正自', '聚精会神', '的', '听', '着', '一个', '瘦削', '的', '老者', '说话', '。', '那', '说话', '人', '五十', '来', '岁', '年纪', '，', '一件', '青布', '长袍', '早洗', '得', '褪成', '了', '蓝灰', '带白', '。', '只', '听', '他', '两片', '梨花', '木板', '碰', '了', '几下', '，', '左手', '中', '竹棒', '在', '一面', '小', '羯鼓', '上', '敲起', '得', '得', '连声', '。', '唱道', '：', '“', '小桃', '无主自', '开花', '，', '烟草', '茫茫', '带', '晚鸦', '。', '几处', '败垣围', '故井', '，', '向来', '一一', '是', '人家', '。', '”', '那', '说话', '人', '将', '木板', '敲', '了', '几下', '，', '说道', '：', '“', '这首', '七言诗', '，', '说', '的', '是', '兵火', '过后', '，', '原来', '的', '家家户户', '，', '都', '变成', '了', '

In [57]:
d2vmodel = gensim.models.Doc2Vec(vector_size=300, window=20, min_count=5)
%time d2vmodel.build_vocab(corp)

Wall time: 2.3 s


In [58]:
d2vmodel.wv.vocab

{'我姓': <gensim.models.keyedvectors.Vocab at 0x1469da90d30>,
 '郭靖自': <gensim.models.keyedvectors.Vocab at 0x146a67aac18>,
 '金国王子': <gensim.models.keyedvectors.Vocab at 0x146a67b4fd0>,
 '几声': <gensim.models.keyedvectors.Vocab at 0x1469ecd5cf8>,
 '所以': <gensim.models.keyedvectors.Vocab at 0x146a77ce0f0>,
 '西方': <gensim.models.keyedvectors.Vocab at 0x146a77eaa20>,
 '结亲': <gensim.models.keyedvectors.Vocab at 0x1469ec434a8>,
 '仰起': <gensim.models.keyedvectors.Vocab at 0x146a77ce128>,
 '叹息': <gensim.models.keyedvectors.Vocab at 0x1469ec43438>,
 '大梁': <gensim.models.keyedvectors.Vocab at 0x146a780c898>,
 '小儿': <gensim.models.keyedvectors.Vocab at 0x1469ec43dd8>,
 '养': <gensim.models.keyedvectors.Vocab at 0x146a7797cf8>,
 '这船': <gensim.models.keyedvectors.Vocab at 0x1469ecd3b00>,
 '妻子': <gensim.models.keyedvectors.Vocab at 0x1469ec43da0>,
 '可见': <gensim.models.keyedvectors.Vocab at 0x146a78112e8>,
 '赶去': <gensim.models.keyedvectors.Vocab at 0x146a7797b70>,
 '三道': <gensim.models.keyedvectors.Voc

In [59]:
# 将新文本转换为相应维度空间下的向量
newvec = d2vmodel.infer_vector(jieba.lcut(chapter.txt[1]))

In [60]:
d2vmodel.docvecs.most_similar([newvec], topn = 10)

[(6, 0.14612214267253876),
 (20, 0.11409661173820496),
 (16, 0.10044573247432709),
 (19, 0.09957955032587051),
 (0, 0.09651108831167221),
 (31, 0.061077699065208435),
 (28, 0.047165341675281525),
 (26, 0.04692249000072479),
 (7, 0.045904457569122314),
 (1, 0.04428883269429207)]

# 文档聚类

In [62]:
# 为章节增加名称标签
chapter.index = [df.txt[df.chap == i].iloc[0] for i in chapter.index]
chapter.head()

,txt,head,mid,len,chap,cut
第一回 风雪惊变,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0,"[第一回, 风雪, 惊变, 钱塘江, 浩浩, 江水, 日日夜夜, 无穷, 无休, 浙西, 临..."
第二回 江南七怪,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0,"[第二回, 江南七怪, 颜烈, 跨出, 房门, 过道, 一个, 中年, 士人, 直响, 一路..."
第三回 黄沙莽莽,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0,"[第三回, 黄沙, 莽莽, 寺里, 僧众, 焦木, 圆寂, 悲哭, 伤者, 包扎, 伤处, ..."
第四回 黑风双煞,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0,"[第四回, 黑风双, 完颜洪熙, 痛快, 蒙古, 前哨, 报来, 王罕, 前来, 迎接, 大..."
第五回 弯弓射雕,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0,"[第五回, 弯弓, 射雕, 一行, 人下, 山来, 不多时, 猛兽, 吼声, 一阵阵, 传来..."


In [63]:
import jieba
cuttxt = lambda x: " ".join(m_cut(x)) 
cleanchap = chapter.txt.apply(cuttxt) 
cleanchap[:2]

第一回 风雪惊变    第一回 风雪 惊变 钱塘江 浩浩 江水 日日夜夜 无穷 无休 浙西 临安 牛家村 绕过 流入...
第二回 江南七怪    第二回 江南七怪 颜烈 跨出 房门 过道 一个 中年 士人 直响 一路打 哈欠 迎面 士人 ...
Name: txt, dtype: object

In [64]:
# 计算TF-IDF矩阵
from sklearn.feature_extraction.text import TfidfTransformer

vectorizer = CountVectorizer() 
wordmtx = vectorizer.fit_transform(cleanchap) # 将文本中的词语转换为词频矩阵  

transformer = TfidfTransformer()  
tfidf = transformer.fit_transform(wordmtx)  #基于词频矩阵计算TF-IDF值  
tfidf

<40x43956 sparse matrix of type '<class 'numpy.float64'>'
	with 129486 stored elements in Compressed Sparse Row format>

In [65]:
# 进行聚类分析
from sklearn.cluster import KMeans  

clf = KMeans(n_clusters = 5)  
s = clf.fit(tfidf)  
print(s)  
clf.cluster_centers_

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00064936,
        0.        ],
       [0.00168883, 0.        , 0.        , ..., 0.        , 0.        ,
        0.00168883],
       [0.        , 0.        , 0.        , ..., 0.00230066, 0.        ,
        0.        ],
       [0.        , 0.00114594, 0.00114594, ..., 0.        , 0.        ,
        0.        ]])

In [66]:
clf.cluster_centers_.shape

(5, 43956)

In [67]:
clf.labels_

array([4, 0, 0, 0, 0, 0, 4, 4, 4, 1, 4, 3, 4, 1, 3, 2, 2, 3, 3, 3, 3, 3,
       1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 2, 2, 2, 3])

In [68]:
chapter['clsres'] = clf.labels_
chapter.head()

,txt,head,mid,len,chap,cut,clsres
第一回 风雪惊变,第一回 风雪惊变钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江...,-224,第钱那几那“他那为若“众村郭小郭张郭张郭张郭张杨杨只隔郭张砰张郭张杨张郭郭杨当便只曲郭郭却曲...,27448,227.0,"[第一回, 风雪, 惊变, 钱塘江, 浩浩, 江水, 日日夜夜, 无穷, 无休, 浙西, 临...",4
第二回 江南七怪,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,-198,第颜颜那颜颜店包包过盖包过盖那包只颜包颜包完完包完突那那完他完那那完这完只完两忽完完完看过和...,25414,402.0,"[第二回, 江南七怪, 颜烈, 跨出, 房门, 过道, 一个, 中年, 士人, 直响, 一路...",0
第三回 黄沙莽莽,第三回 黄沙莽莽寺里僧众见焦木圆寂，尽皆悲哭。有的便为伤者包扎伤处，抬入客舍。忽听得巨钟下的...,-196,第寺忽柯朱法过过丘柯丘朱丘柯柯江丘丘韩丘江丘这韩丘丘丘丘柯全丘韩丘丘韩段段段他李过段李段段每...,22813,597.0,"[第三回, 黄沙, 莽莽, 寺里, 僧众, 焦木, 圆寂, 悲哭, 伤者, 包扎, 伤处, ...",0
第四回 黑风双煞,第四回 黑风双煞完颜洪熙笑道：“好，再打他个痛快。”蒙古兵前哨报来：“王罕亲自前来迎接大金国...,-203,第完沙王王酒铁王完第完哲铁桑完帐铁铁铁哲蒙完完桑桑四赤铁完次白拖那拖那拖拖都四都郭都正都那这...,23324,824.0,"[第四回, 黑风双, 完颜洪熙, 痛快, 蒙古, 前哨, 报来, 王罕, 前来, 迎接, 大...",0
第五回 弯弓射雕,第五回 弯弓射雕一行人下得山来，走不多时，忽听前面猛兽大吼声一阵阵传来。韩宝驹一提缰，胯下黄...,-160,第一韩这郭郭郭朱韩拖郭忽铁完铁都她两铁四铁桑铁完铁全铁江韩江当六全他韩每六漠在朔这韩朱七这南...,20706,815.0,"[第五回, 弯弓, 射雕, 一行, 人下, 山来, 不多时, 猛兽, 吼声, 一阵阵, 传来...",0


In [69]:
chapter.sort_values('clsres').clsres

第二回 江南七怪       0
第三回 黄沙莽莽       0
第四回 黑风双煞       0
第五回 弯弓射雕       0
第六回 崖顶疑阵       0
第二十六回 新盟旧约     1
第二十八回 铁掌峰顶     1
第二十九回 黑沼隐女     1
第三十回 一灯大师      1
第三十一回 鸳鸯锦帕     1
第三十二回 湍江险滩     1
第十四回 桃花岛主      1
第三十四回 岛上巨变     1
第二十三回 大闹禁宫     1
第三十五回 铁枪庙中     1
第十回 往事如烟       1
第二十五回 荒村野店     1
第二十七回 轩辕台前     1
第十六回 《九阴真经》    2
第十七回 双手互搏      2
第三十六回 大军西征     2
第三十九回 是非善恶     2
第三十七回 从天而降     2
第三十八回 锦囊密令     2
第二十回 九阴假经      3
第三十三回 来日大难     3
第四十回 华山论剑      3
第二十二回 骑鲨遨游     3
第二十一回 千钧巨岩     3
第十九回 洪涛群鲨      3
第十八回 三道试题      3
第十五回 神龙摆尾      3
第十二回 亢龙有悔      3
第十三回 五湖废人      4
第十一回 长春服输      4
第九回 铁枪破犁       4
第八回 各显神通       4
第七回 比武招亲       4
第二十四回 密室疗伤     4
第一回 风雪惊变       4
Name: clsres, dtype: int32

In [72]:
chapgrp = chapter.groupby('clsres')
chapcls = chapgrp.apply(sum) # 只有字符串列的情况下，sum函数自动转为合并字符串

cuttxt = lambda x: " ".join(m_cut(x)) 
chapclsres = chapcls.txt.apply(cuttxt) 
chapclsres

clsres
0    第二回 江南七怪 颜烈 跨出 房门 过道 一个 中年 士人 直响 一路打 哈欠 迎面 士人 ...
1    第十回 往事如烟 完颜康 见到 杨铁心 惊诧 之下 认出 大叫 提起 铁枪 行步 朝天 一炷...
2    第十六回 九阴真经 黄二人 自程府 半夜 正想 客店 安歇 马蹄声 骑马 自南 北奔来 渐渐...
3    第十二回 亢龙有悔 黄蓉正 撕开 身后 有人 说道 撕作 三份 鸡屁股 两人 一惊 怎地 背...
4    第一回 风雪 惊变 钱塘江 浩浩 江水 日日夜夜 无穷 无休 浙西 临安 牛家村 绕过 流入...
Name: txt, dtype: object

In [73]:
# 列出关键词以刻画类别特征
import jieba.analyse as ana

ana.set_stop_words(r'D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\停用词.txt')

for item in chapclsres:
    print(ana.extract_tags(item, topK = 10))

['郭靖', '铁木真', '丘处机', '柯镇恶', '韩小莹', '朱聪', '韩宝驹', '梅超风', '完颜洪烈', '段天德']
['黄蓉', '郭靖', '黄药师', '师父', '裘千仞', '梅超风', '黄蓉道', '杨康', '欧阳锋', '武功']
['郭靖', '周伯通', '欧阳锋', '黄蓉', '郭靖道', '成吉思汗', '武功', '裘千仞', '杨康', '丘处机']
['洪七公', '郭靖', '欧阳锋', '黄蓉', '黄药师', '欧阳克', '周伯通', '叫化', '黄蓉道', '武功']
['郭靖', '黄蓉', '完颜康', '杨铁心', '陆冠英', '彭连虎', '包惜弱', '王处一', '丘处机', '侯通海']


## 文本分类 朴素贝叶斯 sklearn

In [74]:
# 从原始语料df中提取出所需的前两章段落
raw12 = df[df.chap.isin([1,2])]
raw12ana = raw12.iloc[list(raw12.txt.apply(len) > 50), :] # 只使用超过50字的段落
raw12ana.reset_index(drop = True, inplace = True)
print(len(raw12ana))
raw12ana.head()

375


,txt,head,mid,len,chap
0,钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江畔一排数十株乌柏...,-1,钱,136,1.0
1,那说话人五十来岁年纪，一件青布长袍早洗得褪成了蓝灰带白。只听他两片梨花木板碰了几下，左手中竹...,-1,那,81,1.0
2,那说话人将木板敲了几下，说道：“这首七言诗，说的是兵火过后，原来的家家户户，都变成了断墙残瓦...,-1,那,342,1.0
3,“叶老汉和叶妈妈吓得呆了，扑将上去，搂住了儿子的尸体，放声大哭。那长官提起狼牙棒，一棒一个，...,-1,“,245,1.0
4,“可是那金兵占了我大宋天下，杀人放火，奸淫掳掠，无恶不作，却又不见他遭到什么报应。只怪我大宋...,-1,“,234,1.0


In [75]:
# 分词和预处理
import jieba

cuttxt = lambda x: " ".join(jieba.lcut(x)) # 这里不做任何清理工作，以保留情感词
raw12ana["cleantxt"] = raw12ana.txt.apply(cuttxt) 
raw12ana.head()

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,txt,head,mid,len,chap,cleantxt
0,钱塘江浩浩江水，日日夜夜无穷无休的从两浙西路临安府牛家村边绕过，东流入海。江畔一排数十株乌柏...,-1,钱,136,1.0,钱塘江 浩浩 江水 ， 日日夜夜 无穷 无休 的 从 两 浙西 路 临安 府 牛家村 边 绕...
1,那说话人五十来岁年纪，一件青布长袍早洗得褪成了蓝灰带白。只听他两片梨花木板碰了几下，左手中竹...,-1,那,81,1.0,那 说话 人 五十 来 岁 年纪 ， 一件 青布 长袍 早洗 得 褪成 了 蓝灰 带白 。 ...
2,那说话人将木板敲了几下，说道：“这首七言诗，说的是兵火过后，原来的家家户户，都变成了断墙残瓦...,-1,那,342,1.0,那 说话 人 将 木板 敲 了 几下 ， 说道 ： “ 这首 七言诗 ， 说 的 是 兵火 ...
3,“叶老汉和叶妈妈吓得呆了，扑将上去，搂住了儿子的尸体，放声大哭。那长官提起狼牙棒，一棒一个，...,-1,“,245,1.0,“ 叶老汉 和 叶 妈妈 吓 得 呆 了 ， 扑 将 上去 ， 搂住 了 儿子 的 尸体 ，...
4,“可是那金兵占了我大宋天下，杀人放火，奸淫掳掠，无恶不作，却又不见他遭到什么报应。只怪我大宋...,-1,“,234,1.0,“ 可是 那金兵 占 了 我 大宋 天下 ， 杀人放火 ， 奸淫掳掠 ， 无恶不作 ， 却 ...


In [76]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer() 

wordmtx = countvec.fit_transform(raw12ana.cleantxt)
wordmtx

<375x6778 sparse matrix of type '<class 'numpy.int64'>'
	with 13758 stored elements in Compressed Sparse Row format>

In [77]:
# 作用：将数据集划分为 训练集和测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(wordmtx, raw12ana.chap, test_size = 0.3, random_state = 111)

In [78]:
from sklearn import naive_bayes

NBmodel = naive_bayes.MultinomialNB()
# 拟合模型
NBmodel.fit(x_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [79]:
x_test

<113x6778 sparse matrix of type '<class 'numpy.int64'>'
	with 4142 stored elements in Compressed Sparse Row format>

In [80]:
NBmodel.predict(x_test)

array([1., 2., 2., 1., 2., 2., 2., 2., 2., 2., 1., 2., 1., 2., 1., 1., 1.,
       2., 1., 1., 1., 2., 2., 1., 2., 1., 1., 2., 1., 1., 1., 1., 1., 2.,
       2., 2., 1., 1., 1., 2., 1., 1., 1., 1., 1., 1., 1., 2., 1., 1., 1.,
       1., 1., 1., 1., 2., 2., 1., 2., 1., 1., 1., 1., 2., 1., 1., 1., 1.,
       1., 2., 1., 1., 2., 2., 1., 1., 1., 2., 2., 1., 2., 1., 2., 2., 2.,
       1., 1., 2., 1., 2., 1., 2., 2., 1., 1., 1., 2., 2., 1., 1., 2., 1.,
       2., 2., 2., 2., 2., 1., 2., 2., 2., 1., 1.])

In [81]:
# 预测准确率（给模型打分）
print('训练集：', NBmodel.score(x_train, y_train), 
      '，验证集：', NBmodel.score(x_test, y_test))

训练集： 1.0 ，验证集： 0.9823008849557522


In [82]:
from sklearn.metrics import classification_report

print(classification_report(y_test, NBmodel.predict(x_test)))

              precision    recall  f1-score   support

         1.0       0.97      1.00      0.98        63
         2.0       1.00      0.96      0.98        50

    accuracy                           0.98       113
   macro avg       0.98      0.98      0.98       113
weighted avg       0.98      0.98      0.98       113



In [86]:
countvec.vocabulary_

{'脱光': 5380,
 '金国王子': 6335,
 '几声': 1296,
 '没饭': 4376,
 '昏君': 3863,
 '实前': 2564,
 '天竺国': 2330,
 '五六岁': 767,
 '小人': 2656,
 '菱肉': 5521,
 '一倒': 21,
 '妻子': 2475,
 '赶去': 5837,
 '视物': 5683,
 '韩世忠': 6568,
 '半句': 1573,
 '并献': 2877,
 '尚未': 2711,
 '密令': 2610,
 '口令': 1720,
 '奋起': 2389,
 '什么': 834,
 '地大口': 2127,
 '南山': 1602,
 '招子': 3458,
 '遭殃': 6199,
 '倘若': 1068,
 '打得众': 3322,
 '熏得': 4576,
 '兄惠然': 1101,
 '瓦罐': 4664,
 '形迹': 2966,
 '对手': 2625,
 '欺蒙': 4225,
 '怀里': 3069,
 '难过': 6512,
 '监寺': 4794,
 '半分': 1572,
 '突转': 5047,
 '枯木': 4140,
 '船里': 5460,
 '之余': 651,
 '一撑': 122,
 '竟是': 5069,
 '万分': 271,
 '全无': 1144,
 '不语': 483,
 '热丧': 4551,
 '月光': 3932,
 '勃然': 1499,
 '遗下': 6191,
 '医道': 1538,
 '牙齿': 4594,
 '官兵': 2544,
 '另划': 1738,
 '发力': 1677,
 '相邀': 4856,
 '颇为': 6593,
 '大畅': 2285,
 '相识': 4850,
 '瘦金体': 4754,
 '永远': 4326,
 '当先': 2942,
 '楼外楼': 4197,
 '秀州': 4991,
 '一并': 86,
 '上空': 323,
 '见个': 5670,
 '口来': 1724,
 '俊雅': 1045,
 '河山': 4378,
 '对饮': 2631,
 '狂叫': 4609,
 '猪肝': 4636,
 '怎受': 3073,
 '花容': 5471,
 '拨落': 3477,


In [87]:
string = "杨铁心和包惜弱收养穆念慈"
words = " ".join(jieba.lcut(string))
words_vecs = countvec.transform([words]) # 数据需要转换为可迭代的list格式
words_vecs

<1x6778 sparse matrix of type '<class 'numpy.int64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [88]:
NBmodel.predict(words_vecs)

array([1.])

## NLTK 朴素贝叶斯

In [91]:
# 这里直接以章节为一个单元进行分析，以简化程序结构
import nltk
from nltk import FreqDist

# 生成完整的词条频数字典，这部分也可以用遍历方式实现
fdist1 = FreqDist(m_cut(chapter.txt[1])) 
fdist2 = FreqDist(m_cut(chapter.txt[2])) 
fdist3 = FreqDist(m_cut(chapter.txt[3])) 
fdist1

FreqDist({'丘处机': 154, '段天德': 55, '完颜洪烈': 51, '柯镇恶': 42, '一个': 39, '焦木': 30, '一声': 29, '贫道': 29, '朱聪': 27, '颜烈': 26, ...})

In [92]:
from nltk.classify import NaiveBayesClassifier

training_data = [ [fdist1, 'chap1'], [fdist2, 'chap2'], [fdist3, 'chap3'] ]

In [93]:
# 训练分类模型
NLTKmodel = NaiveBayesClassifier.train(training_data)

In [94]:
print(NLTKmodel.classify(FreqDist(m_cut("杨铁心收养穆念慈"))))
print(NLTKmodel.classify(FreqDist(m_cut("钱塘江 日日夜夜 包惜弱 颜烈 使出杨家枪"))))

chap2
chap3


In [95]:
nltk.classify.accuracy(NLTKmodel, training_data) # 准确度评价

1.0

In [96]:
NLTKmodel.show_most_informative_features(5)#得到似然比，检测对于哪些特征有用

Most Informative Features
                      浅坑 = None            chap1 : chap3  =      1.0 : 1.0
                      头痛 = None            chap1 : chap3  =      1.0 : 1.0


# 情感分析

In [98]:
# 读入原始数据集
import pandas as pd

dfpos = pd.read_excel(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\购物评论.xlsx", sheet_name = "正向", header=None)
dfpos['y'] = 1
dfneg = pd.read_excel(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\购物评论.xlsx", sheet_name = "负向", header=None)
dfneg['y'] = 0
df0 = dfpos.append(dfneg, ignore_index = True)
df0.head()

,0,y
0,感觉用这个手机下载MP3听很方便，用T-Flash卡装上歌，就可以当一个小MP3用了，很不错...,1
1,外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。...,1
2,我刚拿到书，也没有仔细阅读，就是粗粗的翻了点，觉得还行。里面是蓝黑两种颜色的，有些单词的下面...,1
3,对于二胡曲的精选，应该出版一个系列套装，可分别出售，单独购买。精品二胡曲是有年代和阶段性的，...,1
4,用了一年半的 e680 终于送小偷了。刚刚买了一台e850，用了三天，说说我自己的感受。1：...,1


In [99]:
# 分词和预处理
import jieba

cuttxt = lambda x: " ".join(jieba.lcut(x)) # 这里不做任何清理工作，以保留情感词
df0["cleantxt"] = df0[0].apply(cuttxt) 
df0.head()

,0,y,cleantxt
0,感觉用这个手机下载MP3听很方便，用T-Flash卡装上歌，就可以当一个小MP3用了，很不错...,1,感觉 用 这个 手机 下载 MP3 听 很 方便 ， 用 T - Flash 卡装 上 歌 ...
1,外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。...,1,外观 美观 ， 速度 也 不错 。 上面 一排 触摸 键 挺 实用 。 应该 对得起 这个 ...
2,我刚拿到书，也没有仔细阅读，就是粗粗的翻了点，觉得还行。里面是蓝黑两种颜色的，有些单词的下面...,1,我刚 拿到 书 ， 也 没有 仔细阅读 ， 就是 粗粗 的 翻 了 点 ， 觉得 还 行 。...
3,对于二胡曲的精选，应该出版一个系列套装，可分别出售，单独购买。精品二胡曲是有年代和阶段性的，...,1,对于 二胡曲 的 精选 ， 应该 出版 一个系列 套装 ， 可 分别 出售 ， 单独 购买 ...
4,用了一年半的 e680 终于送小偷了。刚刚买了一台e850，用了三天，说说我自己的感受。1：...,1,用 了 一年 半 的 e680 终于 送 小偷 了 。 刚刚 买 了 一台 e850...


In [100]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer(min_df = 5) # 出现5次以上的才纳入

wordmtx = countvec.fit_transform(df0.cleantxt)
wordmtx

<20582x12014 sparse matrix of type '<class 'numpy.int64'>'
	with 513919 stored elements in Compressed Sparse Row format>

In [101]:
# 按照7：3的比例生成训练集和测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    wordmtx, df0.y, test_size=0.3) # 这里可以直接使用稀疏矩阵格式
x_train[0]

<1x12014 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [102]:
# 使用SVM进行建模
from sklearn.svm import SVC

clf=SVC(kernel = 'rbf', verbose = True)
clf.fit(x_train, y_train) # 内存占用可能较高
clf.score(x_train, y_train)

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]

0.6759214270840563

In [103]:
# 对模型效果进行评估
from sklearn.metrics import classification_report

print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.60      0.93      0.73      3056
           1       0.85      0.40      0.54      3119

    accuracy                           0.66      6175
   macro avg       0.72      0.66      0.64      6175
weighted avg       0.73      0.66      0.64      6175



In [104]:
clf.predict(countvec.transform([df0.cleantxt[0]]))[0]

1

In [105]:
# 模型预测
import jieba

def m_pred(string, countvec, model) : 
    words = " ".join(jieba.lcut(string))
    words_vecs = countvec.transform([words]) # 数据需要转换为可迭代格式
     
    result = model.predict(words_vecs)
    
    if int(result[0]) == 1:
        print(string, "：正向")
    else:
        print(string, "：负向")
        

comment = "外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。风扇噪音也不大。"
m_pred(comment, countvec, clf)

外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。风扇噪音也不大。 ：负向


# 基于分布式表达的情感分析

In [106]:
# 读入原始数据集
import pandas as pd

dfpos = pd.read_excel(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\购物评论.xlsx", sheet_name = "正向", header=None)
dfpos['y'] = 1
dfneg = pd.read_excel(r"D:\资料\教程\NLP基础-网易云课堂\TMData190320\PythonData\购物评论.xlsx", sheet_name = "负向", header=None)
dfneg['y'] = 0
df0 = dfpos.append(dfneg, ignore_index = True)
df0.head()

,0,y
0,感觉用这个手机下载MP3听很方便，用T-Flash卡装上歌，就可以当一个小MP3用了，很不错...,1
1,外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。...,1
2,我刚拿到书，也没有仔细阅读，就是粗粗的翻了点，觉得还行。里面是蓝黑两种颜色的，有些单词的下面...,1
3,对于二胡曲的精选，应该出版一个系列套装，可分别出售，单独购买。精品二胡曲是有年代和阶段性的，...,1
4,用了一年半的 e680 终于送小偷了。刚刚买了一台e850，用了三天，说说我自己的感受。1：...,1


In [107]:
# 分词和预处理，生成list of list格式
import jieba

df0['cut'] = df0[0].apply(jieba.lcut)
df0.head()

,0,y,cut
0,感觉用这个手机下载MP3听很方便，用T-Flash卡装上歌，就可以当一个小MP3用了，很不错...,1,"[感觉, 用, 这个, 手机, 下载, MP3, 听, 很, 方便, ，, 用, T, -,..."
1,外观美观，速度也不错。上面一排触摸键挺实用。应该对得起这个价格。当然再降点大家肯定也不反对。...,1,"[外观, 美观, ，, 速度, 也, 不错, 。, 上面, 一排, 触摸, 键, 挺, 实用..."
2,我刚拿到书，也没有仔细阅读，就是粗粗的翻了点，觉得还行。里面是蓝黑两种颜色的，有些单词的下面...,1,"[我刚, 拿到, 书, ，, 也, 没有, 仔细阅读, ，, 就是, 粗粗, 的, 翻, 了..."
3,对于二胡曲的精选，应该出版一个系列套装，可分别出售，单独购买。精品二胡曲是有年代和阶段性的，...,1,"[对于, 二胡曲, 的, 精选, ，, 应该, 出版, 一个系列, 套装, ，, 可, 分别..."
4,用了一年半的 e680 终于送小偷了。刚刚买了一台e850，用了三天，说说我自己的感受。1：...,1,"[用, 了, 一年, 半, 的, , e680, , 终于, 送, 小偷, 了, 。, ..."


In [108]:
# 按照7：3的比例生成训练集和测试集
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    df0.cut, df0.y, test_size=0.3)
x_train[:2]

6495    [轻, 、, 薄, 绝对, 令人满意, ，, 从, 做工, 来看, 似乎, 也, 能, 延续...
9918    [从, 携程, 上, 预定, 了, 普通, 双人, 标间, ，, 250, /, 晚, ，,...
Name: cut, dtype: object

In [109]:
# 初始化word2vec模型和词表
from gensim.models.word2vec import Word2Vec

n_dim = 300 # 指定向量维度，大样本量时300~500较好

w2vmodel = Word2Vec(size = n_dim, min_count = 10)
w2vmodel.build_vocab(x_train) # 生成词表

In [110]:
# 在评论训练集上建模（大数据集时可能会花费几分钟）
# 本例消耗内存较少
%time w2vmodel.train(x_train, total_examples = w2vmodel.corpus_count, epochs = 10)

Wall time: 5.34 s


(6269298, 9375070)

In [111]:
# 情感词向量间的相似度
w2vmodel.wv.most_similar("不错")

[('好', 0.7261177897453308),
 ('挺不错', 0.6628240942955017),
 ('行', 0.6547620296478271),
 ('很漂亮', 0.6180822253227234),
 ('比较满意', 0.5948976874351501),
 ('也好', 0.5915811061859131),
 ('没得说', 0.5863245725631714),
 ('整体', 0.5699632167816162),
 ('算', 0.5672274827957153),
 ('超值', 0.549899697303772)]

In [112]:
w2vmodel.wv.most_similar("失望")

[('遗憾', 0.6675899028778076),
 ('后悔', 0.6365354061126709),
 ('不负责任', 0.6221133470535278),
 ('气愤', 0.6207784414291382),
 ('牵强', 0.6202349662780762),
 ('生气', 0.6112070679664612),
 ('幼稚', 0.6075685620307922),
 ('不爽', 0.6041831970214844),
 ('难得', 0.5817433595657349),
 ('名不副实', 0.5624862909317017)]

In [113]:
# 生成整句所对应的所有词条的词向量矩阵
pd.DataFrame([w2vmodel.wv[w] for w in df0.cut[0] if w in w2vmodel.wv]).head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-1.026194,-0.509345,-0.321996,0.323929,1.067293,-0.904479,-1.245348,-1.702069,-1.286532,0.699763,...,0.984262,0.784845,-0.986941,-0.833038,-1.550651,1.093480,-0.077803,0.862288,-0.959491,1.097283
1,-0.449001,-0.931802,0.123445,-0.055958,-1.227527,0.631532,-0.084217,-1.011970,0.047239,0.734803,...,1.366179,0.440565,0.886025,0.106636,0.641702,0.044127,-1.063696,0.054665,0.412739,0.316347
2,1.112249,-0.317012,-0.026958,0.068923,-1.179334,0.550494,0.747694,-0.421683,0.648578,-0.956963,...,-0.330753,0.654331,-0.561174,0.289327,0.988951,0.195569,0.090036,-0.461099,-0.819231,-1.386490
3,0.301476,-0.110234,0.418618,-0.634699,0.665380,0.305243,0.340360,-0.325719,-0.699838,0.235860,...,0.918222,-0.082188,-0.047912,-1.007323,1.021879,0.354624,-1.402968,-0.748620,-1.070502,-1.155940
4,0.356840,0.054611,-0.033800,-0.775912,0.232384,0.574802,-0.381796,0.196872,-0.768353,-0.371749,...,1.071766,-0.740354,0.032959,-0.735110,1.070031,0.299830,-0.256355,-0.402975,-0.353711,-0.537746


In [114]:
# 用各个词向量直接平均的方式生成整句对应的向量
def m_avgvec(words, w2vmodel):
    return pd.DataFrame([w2vmodel.wv[w] for w in words if w in w2vmodel.wv]).agg("mean")

In [115]:
# 生成建模用矩阵，耗时较长
%time train_vecs = pd.DataFrame([m_avgvec(s, w2vmodel) for s in x_train])
train_vecs.head()

Wall time: 4min 3s


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.044035,-0.177496,0.000642,0.159698,0.045710,0.149017,-0.156466,-0.262726,-0.239516,0.018026,...,-0.182575,0.247180,-0.121130,-0.262547,0.111498,0.251505,-0.050433,0.006017,-0.051997,-0.365908
1,-0.132313,-0.145055,0.020759,0.044980,0.030540,-0.062094,-0.047027,-0.071491,-0.212660,0.143876,...,-0.158982,0.214030,-0.133005,-0.240921,-0.126407,0.334443,-0.078130,-0.077782,-0.028155,-0.029724
2,-0.087965,-0.161195,-0.014901,0.103302,0.018796,0.222843,-0.259358,-0.094273,-0.067279,0.018595,...,-0.175865,0.031138,-0.262713,-0.224607,0.025357,0.091097,0.001160,0.127400,0.020708,-0.036522
3,-0.458330,-0.207470,0.062975,0.046312,0.204545,-0.217523,-0.009160,0.030941,-0.278020,0.152832,...,-0.357037,0.294484,-0.091376,-0.149222,-0.082085,0.149285,0.128394,-0.107308,-0.113326,-0.125483
4,-0.003180,-0.187051,-0.091965,-0.084898,0.249088,-0.224161,-0.266269,-0.261697,-0.140842,0.196445,...,-0.038352,0.231329,0.132171,-0.443553,0.091338,0.732301,0.214386,-0.035197,0.243614,-0.426509


In [116]:
# 用转换后的矩阵拟合SVM模型
from sklearn.svm import SVC

clf2 = SVC(kernel = 'rbf', verbose = True)
clf2.fit(train_vecs, y_train) # 占用内存小于1G
clf2.score(train_vecs, y_train)

d:\python_install\venv_set\py353\dataanalysiscase\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[LibSVM]

0.8231415284236829

In [117]:
from sklearn.metrics import classification_report

print(classification_report(y_train, clf2.predict(train_vecs))) # 此处未用验证集

              precision    recall  f1-score   support

           0       0.83      0.82      0.82      7188
           1       0.82      0.83      0.82      7219

    accuracy                           0.82     14407
   macro avg       0.82      0.82      0.82     14407
weighted avg       0.82      0.82      0.82     14407



In [118]:
# 模型预测
import jieba

def m_pred(string, model):
    words = jieba.lcut(string)
    words_vecs = pd.DataFrame(m_avgvec(words, w2vmodel)).T
     
    result = model.predict(words_vecs)
    
    if int(result[0]) == 1:
        print(string, "：正向")
    else:
        print(string, "：负向")
        

comment = "作为女儿6.1的礼物。虽然晚到了几天。等拿到的时候，女儿爱不释手，上洗手间也看，告知不好。竟以学习毛主席来反驳我。我反对了几句，还说我对主席不敬。晕。上周末，告诉我她把火鞋和风鞋拿到学校，好多同学羡慕她。呵呵，我也看了其中的人鸦，只可惜没有看完就在老公的催促下睡了。说了这么多，归纳为一句：这套书买的值。"
m_pred(comment, clf2)  

作为女儿6.1的礼物。虽然晚到了几天。等拿到的时候，女儿爱不释手，上洗手间也看，告知不好。竟以学习毛主席来反驳我。我反对了几句，还说我对主席不敬。晕。上周末，告诉我她把火鞋和风鞋拿到学校，好多同学羡慕她。呵呵，我也看了其中的人鸦，只可惜没有看完就在老公的催促下睡了。说了这么多，归纳为一句：这套书买的值。 ：正向


# 文档自动摘要

In [119]:
chapter.txt[1]

'第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过来，那士人似笑非笑，挤眉弄眼，一副惫懒神气，全身油腻，衣冠不整，满面都是污垢，看来少说也有十多天没洗脸了，拿着一柄破烂的油纸黑扇，边摇边行。颜烈见这人衣着明明是个斯文士子，却如此肮脏，不禁皱了眉头，加快脚步，只怕沾到了那人身上污秽，突听那人干笑数声，声音刺耳，经过他身旁时，顺手伸出折扇，往他肩头拍落。颜烈一让，竟没避开，不禁大怒，喝道：“干什么？”那人又是几声干笑，踢跶踢跶地向前去了，只听他走到过道尽头，对店小二道：“喂，伙计啊，你别瞧大爷身上破破烂烂，大爷可有的是银子。有些小子偏邪门着哪，他就是仗着身上光鲜唬人。招摇撞骗，勾引妇女，吃白食，住白店，全是这等小子，你得多留着点儿神。稳稳当当的，让他先交了房饭钱再说。”也不等那店小二答腔，便踢跶踢跶地走了。颜烈更加心头火起，心想好小子，这话不是冲着我来吗？那店小二听那人一说，斜眼向他看了眼，不禁起疑，走到他跟前，哈了哈腰，陪笑道：“您老别见怪，不是小的无礼……”颜烈知他意思，哼了一声道：“把这银子给存在柜上！”伸手往怀里一摸，不禁呆了。他囊里本来放着四五十两银子，一探手，竟已空空如也。店小二见他脸色尴尬，一只手在怀里耽着，摸不出银两，只道穷酸的话不错，神色登时不如适才恭谨，挺腰凸肚地道：“怎么？没带银子吗？”颜烈道：“你等一下，我回房去拿。”他只道匆匆出房，忘拿银两，哪知回入房中打开包裹一看，包里几十两金银尽已不翼而飞。这批金银如何失去，自己竟没半点因头，那倒奇了，寻思：“适才包氏娘子出去解手，我也去了茅房一阵，前后不到一炷香时分，怎地便有人进房来做了手脚？嘉兴府的飞贼倒真厉害。”店小二在房门口探头探脑地张望，见他银子拿不出来，发作道：“这女娘是你原配妻子吗？要是拐带人口，可要连累我们呢！”包惜弱又羞又急，满脸通红。颜烈一个箭步纵到门口，反手一掌，只打得店小二满脸是血，还打落了几枚牙齿。店小二捧住脸大嚷大叫：“好哇！住店不给钱，还打人哪！”颜烈在他屁股上再加一脚，店小二一个筋斗翻了出去。包惜弱惊道：“咱们快走吧，不住这店了。”颜烈笑道：“别怕，没了银子问他们拿。”端了张椅子坐在房门口。过不多时，店小二领了十多名泼皮，抡棍使棒，冲进院子来。颜烈哈哈大笑，喝道：“你们想打架？”忽地跃出，顺手抢过一根杆棒，指东打西，转眼间打倒了四五

In [120]:
def cut_sentence(intxt):  
    delimiters = frozenset('。！？')  
    buf = []  
    for ch in intxt:  
        buf.append(ch)  
        if delimiters.__contains__(ch):  
            yield ''.join(buf)  
            buf = []  
    if buf:  
        yield ''.join(buf) 

In [121]:
sentdf = pd.DataFrame(cut_sentence(chapter.txt[1]))
sentdf

,0
0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...
1,颜烈见这人衣着明明是个斯文士子，却如此肮脏，不禁皱了眉头，加快脚步，只怕沾到了那人身上污秽，...
2,颜烈一让，竟没避开，不禁大怒，喝道：“干什么？
3,”那人又是几声干笑，踢跶踢跶地向前去了，只听他走到过道尽头，对店小二道：“喂，伙计啊，你别瞧...
4,有些小子偏邪门着哪，他就是仗着身上光鲜唬人。
...,...
791,段天德见他来势猛恶，大骇避开。
792,焦木重伤后身法呆滞，竟尔一头撞在大殿柱上，脑浆迸裂，立时毙命。
793,段天德吓得魂不附体，哪里还敢停留，拉了李萍，急奔而出。
794,李萍大叫：“救命啊，我不去，救命啊！


In [122]:
# 去除过短的句子，避免摘要出现无意义的内容
sentdf['txtlen'] = sentdf[0].apply(len)
sentdf.head()

,0,txtlen
0,第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...,116
1,颜烈见这人衣着明明是个斯文士子，却如此肮脏，不禁皱了眉头，加快脚步，只怕沾到了那人身上污秽，...,81
2,颜烈一让，竟没避开，不禁大怒，喝道：“干什么？,23
3,”那人又是几声干笑，踢跶踢跶地向前去了，只听他走到过道尽头，对店小二道：“喂，伙计啊，你别瞧...,64
4,有些小子偏邪门着哪，他就是仗着身上光鲜唬人。,22


In [123]:
sentlist = sentdf[0][sentdf.txtlen > 20]
print(len(sentlist))
sentlist

519


0      第二回 江南七怪颜烈跨出房门，过道中一个中年士人拖着鞋皮，踢跶踢跶的直响，一路打着哈欠迎面过...
1      颜烈见这人衣着明明是个斯文士子，却如此肮脏，不禁皱了眉头，加快脚步，只怕沾到了那人身上污秽，...
2                                颜烈一让，竟没避开，不禁大怒，喝道：“干什么？
3      ”那人又是几声干笑，踢跶踢跶地向前去了，只听他走到过道尽头，对店小二道：“喂，伙计啊，你别瞧...
4                                 有些小子偏邪门着哪，他就是仗着身上光鲜唬人。
                             ...                        
786                    各人见她身穿军士装束，只道是段天德的部属，何以反而拼命拦阻他伤人？
788      柯镇恶眼睛瞎了，耳朵特别灵敏，一听她叫嚷之声，便知是女子，叹道：“焦木和尚，我们都给你害死啦。
790    ”焦木一怔，立时醒悟，心想自己一时不察，给这畜生累死，无意中出卖了良友，又气又急，双手在地上...
792                      焦木重伤后身法呆滞，竟尔一头撞在大殿柱上，脑浆迸裂，立时毙命。
793                          段天德吓得魂不附体，哪里还敢停留，拉了李萍，急奔而出。
Name: 0, Length: 519, dtype: object

In [124]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

txtlist = [ " ".join(jieba.lcut(w)) for w in sentlist]

vectorizer = CountVectorizer() 
X = vectorizer.fit_transform(txtlist) # 将文本中的词语转换为词频矩阵

In [125]:
tfidf_matrix = TfidfTransformer().fit_transform(X)  

In [127]:
# 利用nx包实现pagerank算法
import networkx as nx  

similarity = nx.from_scipy_sparse_matrix(tfidf_matrix * tfidf_matrix.T)
scores = nx.pagerank(similarity)

In [128]:
scores

{0: 0.0016476637063565739,
 1: 0.0020953918806402494,
 2: 0.0021046261921554084,
 3: 0.0015731516752720573,
 4: 0.001344079271965919,
 5: 0.001204033987888288,
 6: 0.0022092098905250974,
 7: 0.0021376064560691983,
 8: 0.0014013541782168584,
 9: 0.0021103815174878257,
 10: 0.0013110998984866547,
 11: 0.0020679834184859996,
 12: 0.0017086463483918664,
 13: 0.0015699317169100167,
 14: 0.002651103721001746,
 15: 0.0014212082442904366,
 16: 0.001183587230914765,
 17: 0.001350625342047038,
 18: 0.001500745636332105,
 19: 0.0012828467276013258,
 20: 0.0013887857355909384,
 21: 0.0014122361002812508,
 22: 0.0018642715383351553,
 23: 0.0014691158969678663,
 24: 0.0021847439118136062,
 25: 0.0021964473173876563,
 26: 0.0015558270422741296,
 27: 0.001475106343808682,
 28: 0.0016620998727064507,
 29: 0.001621203615770347,
 30: 0.0015046311821310483,
 31: 0.0013818622253753247,
 32: 0.001807134223352072,
 33: 0.0022286896822935726,
 34: 0.0016357687678699858,
 35: 0.0021215449716964242,
 36: 0.0012

In [129]:
tops = sorted(scores.items(), key = lambda x: x[1], reverse = True)

In [130]:
tops[:3]

[(131, 0.003613507813534974),
 (510, 0.0035770451197231655),
 (398, 0.0035009935085135277)]

In [132]:
print(sentlist.iloc[tops[0][0]])
print(sentlist.iloc[tops[1][0]])
print(sentlist.iloc[tops[2][0]])

完颜洪烈眼前一花，只见一个道人手托一口极大铜缸，迈步走上楼来，定睛看时，只吓得心中突突乱跳，这道人正是长春子丘处机。
丘处机、焦木和江南七侠武功虽强，这时个个重伤，只有眼睁睁地瞧着他行凶。
两人越说越僵，丘处机一显武功，焦木知道难敌，他与江南七怪素来交好，便约丘处机在醉仙楼上见面。


In [133]:
topn = 5

topsent = sorted(tops[:topn])

abstract = ''
for item in topsent:
    abstract = abstract + sentlist.iloc[item[0]] + "......"
abstract[:-6]

'完颜洪烈眼前一花，只见一个道人手托一口极大铜缸，迈步走上楼来，定睛看时，只吓得心中突突乱跳，这道人正是长春子丘处机。......完颜洪烈心想：“这人眼睛瞎了，又如何接得？......两人越说越僵，丘处机一显武功，焦木知道难敌，他与江南七怪素来交好，便约丘处机在醉仙楼上见面。......”正要差人去请段天德，柯镇恶道：“那丘处机性子好不暴躁，一上来便声势汹汹，浑没把咱们江南武林人物瞧在眼里。......丘处机、焦木和江南七侠武功虽强，这时个个重伤，只有眼睁睁地瞧着他行凶。'

# 自动写作